# Project 101
### PARTE 2.3 - Modelos Redes Neuronales

### Contenido
- [Modelado](#modelos)

    - [**2.3.1 Glounts Deep Ar**](#1)
        - [Definicion del modelo](#def)
        - [Modelado con variable exogena](#exo)
        - [Comparacion de Resultados](#1.3)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import funciones_utiles
from funciones_utiles import DataPrep, Metrics, guardar_metricas

import time
time.clock = time.time

from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.common import ListDataset
from gluonts.torch.model.deepar import DeepAREstimator
from gluonts.torch.distributions import NegativeBinomialOutput
from gluonts.evaluation import make_evaluation_predictions

from sklearn.metrics import mean_squared_error, mean_absolute_error

from dash import Dash, dcc, html, Input, Output
import plotly.graph_objs as go
import plotly.express as px
import dash_bootstrap_components as dbc


__Importamos los df__

In [2]:
comparison_tbl= pd.read_csv('Data/Modelado/comparison_tbl.csv')

In [3]:
df= DataPrep().unificar_df()
df['y']=df['y'].astype(int)
df['ds']= pd.to_datetime(df['ds'])
df.head()

,ds,unique_id,y
0,2019-01-06,impo_1,1950
1,2019-01-13,impo_1,2402
2,2019-01-20,impo_1,2782
3,2019-01-27,impo_1,3263
4,2019-02-03,impo_1,3843


In [4]:
df.columns

Index(['ds', 'unique_id', 'y'], dtype='object')

#### 2.3.1 Glounts Deep Ar


__Definimos el set de train y test__

In [5]:
train = df.loc[df['ds'] < '2022-11-01']
valid = df.loc[df['ds'] >= '2022-11-01']

In [6]:
train_ds = PandasDataset.from_long_dataframe(train,
                                             target='y',
                                             item_id='unique_id', 
                                             timestamp='ds',
                                             freq='W')

__Definicion del modelo DeepAr__

La distribucion por default de DeepAR es un distribucion t-student, pero para modelar los datos es preferible que sea una Negative Binomial ya que permite obtener valores pertenecientes a los Reales Positivos, y no predicciones negativas como ocurre con la distribucion por deafault. Esta informacion es conocidad gracias al domain knowledge que no es posible que hayan valores de kilos negativos sino que pueden tomar valores 0 o positivos. 

In [7]:
tic = time.clock()
estimator = DeepAREstimator(freq='W', #la frecuencia de los datos es semanal por lo que el modelo debe ser semanal tambien
                            context_length=10, # el modelo va a usar las ultimas 10 semanas para predecir las siguientes 10, window de 10
                            prediction_length=10, # va a predecir las proximas 10 semanas 
                            num_layers=4, #el modelo posee 40 capas con un default de 40 nodos por capa 
                            dropout_rate= 0.2, #seteo de 20% de las units en una layer en cero de forma random
                            trainer_kwargs={ 'max_epochs':210},#210 epochs
                            distr_output= NegativeBinomialOutput()) #el default es Distribucion t-student, pero al usar negative binomial permite que las predicciones no tomen valores negativos
                        
predictor = estimator.train(train_ds, num_workers= 2)

toc =  time.clock() #frenamos el cronometro y a continuacion creamos la variable que guarda el tiempo transcurrido desde que empezo a correr el modelo hasta q termino
exetime = '{0:.4f}'.format(toc-tic)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


c:\Users\Bravo15\anaconda3\envs\modelling-torch\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name  | Type        | Params | In sizes | Out sizes   
----------------------------------------------------------------
0 | model | DeepARModel | 50.0 K | ?        | [1, 100, 10]
----------------------------------------------------------------
50.0 K    Trainable params
0         Non-trainable params
50.0 K    Total params
0.200     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Epoch 0, global step 50: 'train_loss' reached 13.45460 (best 13.45460), saving model to 'c:\\Users\\Bravo15\\Desktop\\raico\\Proyecto final\\Project_Raiconet_101\\Experiments\\lightning_logs\\version_10\\checkpoints\\epoch=0-step=50.ckpt' as top 1
Epoch 1, global step 100: 'train_loss' reached 11.00679 (best 11.00679), saving model to 'c:\\Users\\Bravo15\\Desktop\\raico\\Proyecto final\\Project_Raiconet_101\\Experiments\\lightning_logs\\version_10\\checkpoints\\epoch=1-step=100.ckpt' as top 1
Epoch 2, global step 150: 'train_loss' was not in top 1
Epoch 3, global step 200: 'train_loss' was not in top 1
Epoch 4, global step 250: 'train_loss' reached 10.80496 (best 10.80496), saving model to 'c:\\Users\\Bravo15\\Desktop\\raico\\Proyecto final\\Project_Raiconet_101\\Experiments\\lightning_logs\\version_10\\checkpoints\\epoch=4-step=250.ckpt' as top 1
Epoch 5, global step 300: 'train_loss' was not in top 1
Epoch 6, global step 350: 'train_loss' was not in top 1
Epoch 7, global step 400: 't

In [8]:
pred = list(predictor.predict(train_ds))

all_preds = list()
for item in pred:
    unique_id = item.item_id
    p = item.samples.mean(axis=0)
    p10 = np.percentile(item.samples, 10, axis=0)
    p90 = np.percentile(item.samples, 90, axis=0)
    p25 = np.percentile(item.samples, 25, axis=0)
    p75 = np.percentile(item.samples, 75, axis=0)
    dates = pd.date_range(start=item.start_date.to_timestamp(), periods=len(p), freq='W')
    family_pred = pd.DataFrame({'ds': dates, 
                                'unique_id': unique_id,
                                'pred': p,
                                'p25': p25,
                                'p75': p75,
                                'p10': p10, 
                                'p90': p90})
    all_preds += [family_pred]
all_preds = pd.concat(all_preds, ignore_index=True)

all_preds = all_preds.merge(valid, on=['ds', 'unique_id'], how='left')

all_preds.dropna(inplace=True)

Metrics(all_preds['y'], all_preds['pred']).calculate_smape()

0.646356141274478

In [9]:
all_preds.head()

,ds,unique_id,pred,p25,p75,p10,p90,y
0,2022-11-06,expo_1,1644.780029,1114.25,2046.50,835.4,2491.4,2720.0
1,2022-11-13,expo_1,1739.910034,1205.25,2266.00,911.5,2706.8,290.0
2,2022-11-20,expo_1,1621.199951,1188.25,1908.50,931.5,2428.4,906.0
3,2022-11-27,expo_1,1651.510010,1219.25,1975.75,924.2,2409.0,2408.0
4,2022-12-04,expo_1,1625.819946,1027.75,2106.50,830.0,2545.8,326.0
...,...,...,...,...,...,...,...,...
114,2022-12-04,impo_7,230.809998,139.75,294.50,97.0,381.5,278.0
115,2022-12-11,impo_7,193.300003,97.75,261.25,67.0,371.2,182.0
116,2022-12-18,impo_7,231.160004,149.75,293.00,63.9,374.5,260.0
117,2022-12-25,impo_7,197.389999,112.50,263.50,56.9,342.5,409.0


__Resultados Obtenidos: Deep AR__

In [10]:
app = Dash(external_stylesheets=[dbc.themes.SLATE])



dropdown = dcc.Dropdown(
    id="ticker",
    options=[{"label": unique_id, "value": unique_id} for unique_id in df["unique_id"].unique()],
    value="impo_1",
    clearable=False, 
    style={'background-color': 'grey'}
)

app.layout = html.Div([
    html.H4('Deep AR Resultados'),
    html.P("Seleccionar motivo:"),
    dropdown,
    dcc.Graph(id="time-series-chart"),
])


@app.callback(
    Output("time-series-chart", "figure"), 
    Input("ticker", "value"))

def display_time_series(ticker):
    df_pivot = df.pivot(index='ds', columns='unique_id', values='y')
    df_pivot = df_pivot.reset_index()
    df_pivot = df_pivot.fillna(0)
    
    fig = px.line(df_pivot, x='ds', y=ticker, title=ticker, template='plotly_dark')
    fig.update_layout(plot_bgcolor='rgba(0, 0, 0, 0)',
                      paper_bgcolor='rgba(0, 0, 0, 0)')
    
    #bandas de percentiles 
    p_ = all_preds.loc[all_preds['unique_id'] == ticker]
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p10'], mode='lines', line=dict(color='rgba(255, 165, 0, 0.2)'), name='p10'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p90'], mode='lines', line=dict(color='rgba(255, 165, 0, 0.2)'), name='p90', fill='tonexty', fillcolor='rgba(255, 165, 0, 0.2)'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p25'], mode='lines', line=dict(color='orange'), name='p25'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p75'], mode='lines', line=dict(color='orange'), name='p75', fill='tonexty', fillcolor='rgba(255, 165, 0, 0.2)'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['pred'], mode='lines', line=dict(color='orange'), name='Forecast'))
    return fig


if __name__ == "__main__":
    app.run_server(debug=True, port=8080, mode='inline')

__Guardamos las métricas para cada una de las series__

In [11]:
comparison_tbl=guardar_metricas(comparison_tbl, all_preds, exetime, 'Glounts Deep AR')

In [12]:
comparison_tbl.tail(12)

,Serie,Modelo,MAE,RMSE,sMAPE,Processing Time
83,expo_1,Glounts Deep AR,903.663317,963.275835,0.730640,1233.5014
84,expo_2,Glounts Deep AR,773.753338,967.771505,1.530043,1233.5014
85,expo_3,Glounts Deep AR,173.085999,177.500763,0.497804,1233.5014
86,expo_4,Glounts Deep AR,106.772223,122.042055,0.883213,1233.5014
87,expo_5,Glounts Deep AR,55.542225,72.673071,0.337723,1233.5014
88,impo_1,Glounts Deep AR,1106.998888,1277.032392,0.345353,1233.5014
89,impo_2,Glounts Deep AR,75.781104,93.637130,0.187901,1233.5014
90,impo_3,Glounts Deep AR,27.428889,29.240702,1.151015,1233.5014
91,impo_4,Glounts Deep AR,49.663333,59.154266,0.663286,1233.5014
92,impo_5,Glounts Deep AR,561.704434,874.288431,0.539238,1233.5014


#### 2.3.1 Glounts Deep AR - Variable exógena

Cotizacion dolar oficial

Obtenida de: https://es.investing.com/currencies/usd-ars-historical-data

In [13]:
df_cot_dolar_blue= pd.read_csv('Data/Datos históricos USD_ARSB.csv')
df_cot_dolar_blue['Fecha'] = pd.to_datetime(df_cot_dolar_blue['Fecha'], format='%d.%m.%Y')
df_cot_dolar_blue= df_cot_dolar_blue[['Fecha','Último']]
df_cot_dolar_blue['Último']= df_cot_dolar_blue['Último'].str.replace(',', '.')
df_cot_dolar_blue['Último']= df_cot_dolar_blue['Último'].astype(float)

In [14]:
df_cot_dolar_blue.tail()

,Fecha,Último
208,2019-02-03,37.38
209,2019-01-27,37.62
210,2019-01-20,38.12
211,2019-01-13,38.88
212,2019-01-06,39.00


In [15]:
df_merged = df.merge(df_cot_dolar_blue, left_on='ds', right_on='Fecha', how='left')
df_merged = df_merged.rename(columns={'Último': 'cotizacion'}).drop('Fecha', axis=1)
df_merged['cotizacion']=df_merged['cotizacion'].astype(int)

In [16]:
df_merged.head()

,ds,unique_id,y,cotizacion
0,2019-01-06,impo_1,1950,39
1,2019-01-13,impo_1,2402,38
2,2019-01-20,impo_1,2782,38
3,2019-01-27,impo_1,3263,37
4,2019-02-03,impo_1,3843,37


__Definimos el set de train y test__

In [17]:
train= df_merged
valid = df_merged[df_merged['ds'] >= '2022-11-01']

#cuando agregamos una variable exogena la misma debe ser input en la fase de test, es decir debe ser una variable conocida a futuro.
#Aunque en la practica es casi inviable conocer la cotizacion del dolar a futuro se podria crear un segundo modelo que prediga la misma, 
#pero en esta instancia a modo de prueba correre el modelo bajo el supuesto que podremos agregar la cotizacion del dolar futuro en el modelo 
#una vez que este en produccion

In [18]:
train_ds = PandasDataset.from_long_dataframe(train,
                                             target='y', 
                                             item_id='unique_id', 
                                             timestamp='ds',
                                             freq='W', 
                                             feat_dynamic_real=['cotizacion']) #agregamos la variable exogena 
#future_lenght = 10 
#pasarle al train todo el dataset, la variable target hasta el 1 de noviembre, y necesitos 10 valores de cot hacia el futuro 
#como es autoregresivo necesita los valores futuros, valor del dolar hasta hoy 
#prediga las dos variables -> deep VAR -> predecir las dos cosas, target + exogenas => estimar por fuera el dolar -> input nuevo modelo 
#hacer la prueba si realmente mejora evaluar las opciones => el valor observado futuro nunca lo voy a tener -> valor estimado != valor futuro 
#variable categorica estatica => pais de origen del motivo -> USA, EUROPA, CHINA etc -> static categorical no varia en el tiempo

__Definicion del Modelo DeepAr - feat dynamic real__

In [19]:
tic = time.clock()
estimator2 = DeepAREstimator(freq='W', #la frecuencia de los datos es semanal por lo que el modelo debe ser semanal tambien
                            context_length=9, # el modelo va a usar las ultimas 9 semanas para predecir la siguientes 10
                            prediction_length=9, # va a predecir las proximas 9 semanas 
                            num_layers=4, #el modelo posee 4 capas con un default de 40 nodos por capa 
                            dropout_rate= 0.2, #seteo de 20% de las units en una layer en cero de forma random, el default es 0.1
                            trainer_kwargs={ 'max_epochs':10},#120, #en vez de poner un early stopping defino un max epoch de 30 ((210)
                            num_feat_dynamic_real= int(1), 
                            distr_output= NegativeBinomialOutput())  #el default es Distribucion t-student, pero al usar negative binomial permite que las predicciones no tomen valores negativos

predictor2 = estimator2.train(train_ds, num_workers=2)

toc =  time.clock() #frenamos el cronometro y a continuacion creamos la variable que guarda el tiempo transcurrido desde que empezo a correr el modelo hasta q termino
exetime = '{0:.4f}'.format(toc-tic)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\Bravo15\anaconda3\envs\modelling-torch\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:108: PossibleUserWarning:

You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.


  | Name  | Type        | Params | In sizes | Out sizes  
---------------------------------------------------------------
0 | model | DeepARModel | 50.2 K | ?        | [1, 100, 9]
---------------------------------------------------------------
50.2 K    Trainable params
0         Non-trainable params
50.2 K    Total params
0.201     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Epoch 0, global step 50: 'train_loss' reached 15.95339 (best 15.95339), saving model to 'c:\\Users\\Bravo15\\Desktop\\raico\\Proyecto final\\Project_Raiconet_101\\Experiments\\lightning_logs\\version_11\\checkpoints\\epoch=0-step=50.ckpt' as top 1
Epoch 1, global step 100: 'train_loss' was not in top 1
Epoch 2, global step 150: 'train_loss' reached 15.11424 (best 15.11424), saving model to 'c:\\Users\\Bravo15\\Desktop\\raico\\Proyecto final\\Project_Raiconet_101\\Experiments\\lightning_logs\\version_11\\checkpoints\\epoch=2-step=150.ckpt' as top 1
Epoch 3, global step 200: 'train_loss' was not in top 1
Epoch 4, global step 250: 'train_loss' reached 11.54418 (best 11.54418), saving model to 'c:\\Users\\Bravo15\\Desktop\\raico\\Proyecto final\\Project_Raiconet_101\\Experiments\\lightning_logs\\version_11\\checkpoints\\epoch=4-step=250.ckpt' as top 1
Epoch 5, global step 300: 'train_loss' was not in top 1
Epoch 6, global step 350: 'train_loss' reached 10.32187 (best 10.32187), saving mode

In [20]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=train_ds,  
    predictor=predictor2, 
)

tss = list(ts_it)
forecasts = list(forecast_it)

In [21]:
valid.head()

,ds,unique_id,y,cotizacion
200,2022-11-06,impo_1,3067,291
201,2022-11-13,impo_1,2557,304
202,2022-11-20,impo_1,1651,318
203,2022-11-27,impo_1,2538,310
204,2022-12-04,impo_1,2321,314


In [22]:
all_preds2 = list()
for item in forecasts:
    unique_id = item.item_id
    p = item.samples.mean(axis=0)
    p10 = np.percentile(item.samples, 10, axis=0)
    p90 = np.percentile(item.samples, 90, axis=0)
    p25 = np.percentile(item.samples, 25, axis=0)
    p75 = np.percentile(item.samples, 75, axis=0)
    dates = pd.date_range(start=item.start_date.to_timestamp(), periods=len(p), freq='W')
    family_pred = pd.DataFrame({'ds': dates, 
                                'unique_id': unique_id,
                                'pred': p,
                                'p25': p25,
                                'p75': p75,
                                'p10': p10, 
                                'p90': p90})
    all_preds2 += [family_pred]
all_preds2 = pd.concat(all_preds2, ignore_index=True)
all_preds2 = all_preds2.merge(valid, on=['ds', 'unique_id'], how='left')

all_preds2.dropna(inplace=True)

Metrics(all_preds2['y'], all_preds2['pred']).wmape()

0.5434404768405177

__Resultados Obtenidos: Deep AR - feat dynamic real__

In [23]:
app2 = Dash(external_stylesheets=[dbc.themes.SLATE])


dropdown = dcc.Dropdown(
    id="ticker",
    options=[{"label": unique_id, "value": unique_id} for unique_id in df["unique_id"].unique()],
    value="impo_1",
    clearable=False, 
    style={'background-color': 'grey'}
)

app2.layout = html.Div([
    html.H4('Deep AR - Feat dynamic real Resultados'),
    html.P("Seleccionar motivo:"),
    dropdown,
    dcc.Graph(id="time-series-chart"),
])


@app2.callback(
    Output("time-series-chart", "figure"), 
    Input("ticker", "value"))

def display_time_series(ticker):
    df_pivot = df.pivot(index='ds', columns='unique_id', values='y')
    df_pivot = df_pivot.reset_index()
    df_pivot = df_pivot.fillna(0)
    
    fig = px.line(df_pivot, x='ds', y=ticker, title=ticker, template='plotly_dark')
    fig.update_layout(plot_bgcolor='rgba(0, 0, 0, 0)',
                      paper_bgcolor='rgba(0, 0, 0, 0)')
    
    #bandas de percentiles 
    p_ = all_preds2.loc[all_preds2['unique_id'] == ticker]
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p10'], mode='lines', line=dict(color='rgba(255, 165, 0, 0.2)'), name='p10'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p90'], mode='lines', line=dict(color='rgba(255, 165, 0, 0.2)'), name='p90', fill='tonexty', fillcolor='rgba(255, 165, 0, 0.2)'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p25'], mode='lines', line=dict(color='orange'), name='p25'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['p75'], mode='lines', line=dict(color='orange'), name='p75', fill='tonexty', fillcolor='rgba(255, 165, 0, 0.2)'))
    fig.add_trace(go.Scatter(x=p_['ds'], y=p_['pred'], mode='lines', line=dict(color='orange'), name='Forecast'))
    return fig


if __name__ == "__main__":
    app2.run_server(debug=True, port=8081, mode='inline')

__Guardamos las métricas para cada una de las series__

In [24]:
comparison_tbl=guardar_metricas(comparison_tbl, all_preds, exetime, 'Glounts Deep AR feat dynamic variable')

In [25]:
comparison_tbl.tail(12)

,Serie,Modelo,MAE,RMSE,sMAPE,Processing Time
95,expo_1,Glounts Deep AR feat dynamic variable,903.663317,963.275835,0.730640,76.2638
96,expo_2,Glounts Deep AR feat dynamic variable,773.753338,967.771505,1.530043,76.2638
97,expo_3,Glounts Deep AR feat dynamic variable,173.085999,177.500763,0.497804,76.2638
98,expo_4,Glounts Deep AR feat dynamic variable,106.772223,122.042055,0.883213,76.2638
99,expo_5,Glounts Deep AR feat dynamic variable,55.542225,72.673071,0.337723,76.2638
100,impo_1,Glounts Deep AR feat dynamic variable,1106.998888,1277.032392,0.345353,76.2638
101,impo_2,Glounts Deep AR feat dynamic variable,75.781104,93.637130,0.187901,76.2638
102,impo_3,Glounts Deep AR feat dynamic variable,27.428889,29.240702,1.151015,76.2638
103,impo_4,Glounts Deep AR feat dynamic variable,49.663333,59.154266,0.663286,76.2638
104,impo_5,Glounts Deep AR feat dynamic variable,561.704434,874.288431,0.539238,76.2638


**Conclusion:**  A pesar de agregar la variable exogena del tipo de cambio, aunque los resultados del modelo no mejoran y aun el modelo Glounts Deep AR permite obtener mejores predicciones. Comparando los resultados entre el primer modelo Deep Ar y el segundo Deep Ar con feat dynamic real, se observa que ambos no logran captar las tendencias y cambios drasticos existentes en los datos. Si bien los resultados son mejores que los obtenidos con los modelos clasicos, los modelos de machine learning con entrenamiento global poseen mejores metricas, aun asi el modelo permite obtener predicciones con cierto margen de error pero aun no logra captar los cambios abruptos que poseen las series. En gran medida se debe a la naturaleza del modelo Deep AR, ya que como su nombre indica al ser un modelo autoregresivo busca patrones en valores historicos de la serie buscando hacer inferencias con distribuciones conocidas. Pero aun asi, como los datos presentes son erraticos y no revelan una distribucion aparente este tipo de modelo no parece ser suficiente. 